task: https://www.coursera.org/learn/supervised-learning/programming/lWdVx/beghghingh-i-sluchainyi-lies

# Бэггинг и случайный лес

In [1]:
def write_ans(file_name, ans):
    with open(file_name, "w") as fout:
        fout.write(str(ans))

In [2]:
from sklearn import ensemble, model_selection, metrics, tree

import numpy as np
import pandas as pd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from sklearn.datasets import load_digits

digits_dataset = load_digits()
X = digits_dataset.data
y = digits_dataset.target
print(digits_dataset.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

Из описания или с помощью 'digits_dataset.data.shape' понимаем что раотаем с 64 признаками.

In [4]:
n_features = X.shape[1] # задали количество признаков
X, y

(array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]), array([0, 1, 2, ..., 8, 9, 8]))

## Пункт 1

Создадим класс DecisionTreeClassifier и измерим качество его работы с помощью cross_val_score

In [5]:
clf = tree.DecisionTreeClassifier(random_state=0)
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [6]:
from sklearn.model_selection import cross_val_score

cvs = cross_val_score(estimator=clf, X = X, y = y, cv=10)
print('Ответ на пункт 1:', cvs.mean())

Ответ на пункт 1: 0.8241247672253259


In [7]:
write_ans("answer1.txt", cvs.mean())

## Пункт 2

Обучим бэггинг над DecisionTreeClassifier. Зададим количество деревьев равным 100.

In [8]:
from sklearn.ensemble import BaggingClassifier

clf_bagging = BaggingClassifier(base_estimator= clf,
                        n_estimators=100, random_state=0).fit(X, y)

cvs = cross_val_score(estimator=clf_bagging, X = X, y = y, cv=10)

print('Ответ на пункт 2:', cvs.mean())

Ответ на пункт 2: 0.9253786468032278


**Вывод:** Качество работы композиции решающих деревьев заметно выросло по сранвнеию с одним решающим деревом.

In [9]:
write_ans("answer2.txt", cvs.mean())

## Пункт 3
Добьёися того, чтобы каждый базовый алгоритм BaggingClassifier обучался не на всех d признаках, а на $\sqrt{d}$. Для этого будем использовать параметр `max_features`.

In [10]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [11]:
clf_bagging = BaggingClassifier(base_estimator= clf,
                        n_estimators=100, random_state=0, max_features=int(np.sqrt(n_features))).fit(X, y)

cvs = cross_val_score(estimator=clf_bagging, X = X, y = y, cv=10)
print('Ответ на пункт 3:', cross_val_score(estimator=clf_bagging, X = X, y = y, cv=10).mean())

Ответ на пункт 3: 0.9304283054003726


**Вывод:** Качество еще выросло.

In [12]:
write_ans("answer3.txt", cvs.mean())

## Пункт 4

Попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Для этого нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier.
(за это отвечает все тот же параметр `max_features` в классе `DecisionTreeClassifier`)

In [13]:
clf = tree.DecisionTreeClassifier(random_state=0, max_features=int(np.sqrt(n_features)))
clf_bagging = BaggingClassifier(base_estimator= clf,
                        n_estimators=100, random_state=0).fit(X, y)

cvs = cross_val_score(estimator=clf_bagging, X = X, y = y, cv=10)
print('Ответ на пункт 4:', cvs.mean())

Ответ на пункт 4: 0.9510211049037863


**Вывод:** качество выросло 

In [14]:
write_ans("answer4.txt", cvs.mean())

## Пункт 5
1. Поймем что бэггинг на рандомизированных деревьях соответствует алгоритму Random Forest. Сравним их качество
2. Посмотрим как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева (построим соответствующие графики)
3. Выберем верные утверждения из следующего списка:

    1) Случайный лес сильно переобучается с ростом количества деревьев
    
    2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев
    
    3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
    
    4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
    
    5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.
    
    6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.
    
    7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
%%time
rf_classifier = RandomForestClassifier(random_state=0)
bagging = BaggingClassifier(rf_classifier, n_estimators=100)

cross_val_score(bagging, X, y, cv = 10).mean()

CPU times: user 3min 39s, sys: 2.23 s, total: 3min 41s
Wall time: 3min 41s


0.9482278088144008

In [17]:
# действительно ли тоже самое получилось?
# обратить внимание на время обучения

In [18]:
import tqdm
from tqdm import tnrange
from sklearn.model_selection import cross_val_score, validation_curve

In [19]:
# зададим параметры для посотроения зависимости качества модели от количества дерьев,
# количества признаков, огрнаичении на глубину дерева

n_estimators_range = np.array([1] + [i for i in range(10,101, 10)])
max_features_range = np.array([5,10,20,50, 64])
max_depth_range = np.array([i for i in range(5,100, 5)])

Будем пользоваться validation_curve для определния качества полченной модели

(не learning_curve потому что подбираем гиперпараметры, а не *training_example*)

In [ ]:
%%time

# для n_estimators_range
train_scores, test_scores = validation_curve(bagging, X, y, param_name="n_estimators", 
                                             param_range=n_estimators_range, cv=10, scoring="accuracy")

In [ ]:
pylab.grid(True)
pylab.title('Зависимость качество на тренировочной и тествой выборки взависимости от количества дереьев')
pylab.plot(n_estimators_range, train_scores.mean(axis = 1), 'g-', marker='o', label='train')
pylab.plot(n_estimators_range, test_scores.mean(axis = 1), 'r-', marker='o', label='test')
pylab.legend(loc='lower right')

In [ ]:
%%time

# для max_features_range
train_scores_features, test_scores_features = validation_curve(bagging, X, y, param_name="max_features", 
                                             param_range=max_features_range, cv=10, scoring="accuracy")

In [ ]:
pylab.grid(True)
pylab.title('Зависимость качество на тренировочной и тествой выборки взависимости от числа признаков')
pylab.plot(max_features_range, train_scores_features.mean(axis = 1), 'g-', marker='o', label='train')
pylab.plot(max_features_range, test_scores_features.mean(axis = 1), 'r-', marker='o', label='test')
pylab.legend(loc='lower right')

In [ ]:
%%time

# для max_depth_range
train_scores_depth, test_scores_depth = validation_curve(bagging, X, y, param_name="base_estimator__max_depth", 
                                             param_range=max_features_range, cv=10, scoring="accuracy")

In [ ]:
pylab.grid(True)
pylab.title('Зависимость качество на тренировочной и тествой выборки взависимости от числа макисмальной глубины дерева')
pylab.plot(max_features_range, train_scores_depth.mean(axis = 1), 'g-', marker='o', label='train')
pylab.plot(max_features_range, test_scores_depth.mean(axis = 1), 'r-', marker='o', label='test')
pylab.legend(loc='lower right')

**Итого:** Верные утверждения из списка: 
2 3 4 7

In [ ]:
write_ans("answer5.txt", '2 3 4 7')

# TODO: 
1. разобраться почему 4 правильный а не 5!
2. Понять в чем разница юзать `validation_curve` или просто итерироваться по массиву с параметрами и на каждом этапе считать cross_val_score